In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Primer Cuatrimestre de 2018. Examen parcial, primera oportunidad.

Nintendo of America (EEUU) tiene
información de ventas de videojuegos
físicas mensuales totalizadas en EEUU
las cuales se realizan en cadenas de
tiendas de videojuegos en el siguiente
RDD: (id_videojuego, id_tienda, mes,
anio, total_ventas_mensuales).

Por otro lado tenemos un RDD con
información de las tiendas y de su
ubicación (id_tienda, direccion, latitud,
longitud, codigo_postal, estado).
Con esta información escribir un
programa en pySpark para obtener la
tienda que realizó menor cantidad de
ventas en el estado de "Georgia" en todo
el año 2017. 

Criterio: Es importante filtrar los datos que son necesarios antes de comenzar a trabajar, si no lo hacen se descuenta un min de 5ptos. Hay descuentos de 3
ptos si realizan operaciones de mas, o ineficientes (por ejemplo realizar un takeordered cuando necesitan solo obtener mínimo).
Si los formatos para realizar el join no se corresponde a (K, V) descuento de 5 puntos. 

In [39]:
sales = [
    (1, 1, '01', '2017', 500), # sera la minima
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 1, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '02', '2017', 500),
    (4, 3, '03', '2017', 500),    

]

stores = [
    (1 , 'address 1', -1, -1, '30002', 'Georgia'),
    (2 , 'address 2', -2, -2, '30003', 'Georgia'),
    (3 , 'address 2', -3, -3, '30004', 'Georgia'),
    (4 , 'address 2', -4, -4, '10119', 'New York')    
]

In [40]:
# Obtener la tienda que realizó menor cantidad de ventas en el estado de "Georgia" en todo el año 2017

In [41]:
salesRdd = sc.parallelize(sales)
storesRdd = sc.parallelize(stores)

In [42]:
# filtramos la informacion de 2017 de ventas y realizamos un cambio de clave
salesToJoin = salesRdd.filter(lambda x: x[3] == '2017')\
    .map(lambda x: (x[1], (x[1],x[4]))) # (K,V) K: id_tienda, V: (id_tienda, total_ventas_mensuales)

# consideramos solo tiendas de Georgia
storesToJoin = storesRdd.filter(lambda x: x[5] == 'Georgia')\
    .map(lambda x: (x[0], x[0]))

In [50]:
# realizamos el join de ambos data frames
# simplificamos el formato para evitar los reduce complejos
# sumarizamos cantidad de ventas por tienda
# buscamos el minimo con reduce
salesToJoin.join(storesToJoin)\
    .map(lambda x: (x[0], x[1][0][1]))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if x[1] < y[1] else y)

(1, 500)

## Primer Cuatrimestre de 2018. Examen parcial, tercera oportunidad

El GCPD (Gotham City Police Dept) recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un archivo con el siguiente formato: (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).

Los posibles estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolución. Las fechas se encuentran en el formato YYYY-MM-DD. 

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuáles casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respuesta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.
El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a)  Las categorías que hayan incrementado su tasa de resolución en al menos un 10%  en el último trimestre, con respecto al trimestre anterior.
b) Tasa de participación de Batman por categoría, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto, y robo de vehículos) 

Resolución:

Primero filter por fecha ult dos trimestres. Luego map con key compuesta categoria y trimestre y value (estado_caso == resuelto, estado == sin_resolucion or estado == resuelto).  ReduceByKey, donde sumes los dos values. Un map cambiando la clave a categoria, y dejando en value el trimestre y los dos valores. Un groupByKey para juntar en la misma categoria los dos valores de cada trimestre. Por ultimo un filter que compare las tasas de cada trimestre, para ver cuales cumplen con la condición. Collect al final. Vale 8 puntos. 

Si no filtran al ppio descuento 3 puntos. Map u operaciones innecesarias descuento de 3 puntos mínimo.

Otra solución posible es: se pueden generar dos RDD: uno para cada trimestre. Sobre esos dos RDD, se calculan las tasas de resolución por categoría y luego se los joinea por el campo de categoría para poder verificar la condición de que haya incrementado la resolución en un 10%. Esta segunda alternativa no es escalable, ya que si tuvieramos que trabajar con mas trimestres o hacer el corte por mes o por dia no tendria sentido generar un rdd por dia. (Descuento de 5puntos)

b) 	Será necesario primero filtrar el RDD por las categorías necesarias para reducir el volumen de información (descuento de 3 puntos si lo hacen después) y luego joinear por id_caso con el segundo RDD utilizando un left join (descuento de 3 puntos si se usa inner, 5 puntos si las claves no coinciden, no sirve). El left join nos dará aquellos casos en los que batman fue llamado y en los que no fue llamados. A partir de ahí, podemos mapear algo del estilo (categoria, (fue_llamado, 1)) y con un reduceByKey calcular la tasa de participación. El campo “fue_llamado” será 1 si respondió a la batiseñal o 0 en caso contrario. El b vale 7 puntos.

In [201]:
# (fecha, id_caso, descripción, estado_caso, categoría, latitud, longitud).
cases = [("2019-01-01", 1, "case 1", 2, "otro delito", -1, -1), 
         ("2019-06-01", 2, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 3, "case 2", 3, "robo", -1, -1),         
         ("2019-06-01", 4, "case 2", 1, "robo", -1, -1),         
         ("2019-06-01", 5, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 6, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 9, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 10, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 60, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 70, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 80, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 90, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 100, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 600, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 700, "case 2", 3, "robo", -1, -1),         
         ("2019-09-01", 800, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 900, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 1000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 6000, "case 2", 2, "robo", -1, -1),
         ("2019-09-01", 7000, "case 2", 2, "robo", -1, -1),         
         ("2019-09-01", 8000, "case 2", 3, "robo", -1, -1),
         ("2019-09-01", 9000, "case 2", 1, "robo", -1, -1),
         ("2019-09-01", 10000, "case 2", 2, "robo", -1, -1),
         ("2019-06-01", 92, "case 2", 2, "hurto", -1, -1),
         ("2019-06-01", 93, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 94, "case 2", 3, "hurto", -1, -1),         
         ("2019-06-01", 95, "case 2", 3, "hurto", -1, -1),
         ("2019-09-01", 96, "case 2", 2, "hurto", -1, -1),
 
        ]

# (id_caso, respuesta)
batsignal = [(1,1),
         (2,1),
         (3,0),
         (4,0),
         (5,1),
         (6,0),
         (7,1),
         (8,0),
         (9,1),
         (10,0),         
         (60,0),
         (70,1),
         (80,1),
         (90,1),
         (100,1),
         (600,0),
         (700,1),
         (800,0),
         (900,1),
         (1000,1),
         (6000,0),
         (7000,1),
         (8000,0),
         (9000,1),
         (10000,1),
         (92,0),
         (93,0),             
         (94,0),
         (95,0),             
         (96,1)             
        ]

In [202]:
casesRdd = sc.parallelize(cases)
batsignalRdd = sc.parallelize(batsignal)

In [203]:
#a) Las categorías que hayan incrementado su tasa de resolución en al menos un 10% en el último trimestre, con respecto al trimestre anterior.

# filtramos datos de los trimestres correspondientes
cases_filtered = casesRdd.filter(lambda x: (x[0] >= '2019-04-00' and x[0] < '2019-10-00'))
#cases_filtered.collect()

In [204]:
def generate_key(x):
    term = 3
    if ((x[0] >= '2019-04-00') and (x[0] < '2019-07-00')):
        term = 2
    
    # K: (categoria, numero_trimestre)
    # V: (estado_caso == resuelto, estado == sin_resolucion or estado == resuelto)
    return ((x[4],term), (int(x[3] == 2), int((x[3] == 3 or x[3] == 2))))

In [205]:
cases_per_cat_term = cases_filtered.map(generate_key)
cases_per_cat_term.collect()

[(('robo', 2), (1, 1)),
 (('robo', 2), (0, 1)),
 (('robo', 2), (0, 0)),
 (('robo', 2), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (0, 1)),
 (('robo', 3), (0, 1)),
 (('robo', 3), (0, 1)),
 (('robo', 3), (0, 0)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (0, 1)),
 (('robo', 3), (0, 0)),
 (('robo', 3), (0, 0)),
 (('robo', 3), (0, 0)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (1, 1)),
 (('robo', 3), (0, 1)),
 (('robo', 3), (0, 0)),
 (('robo', 3), (1, 1)),
 (('hurto', 2), (1, 1)),
 (('hurto', 2), (0, 1)),
 (('hurto', 2), (0, 1)),
 (('hurto', 2), (0, 1)),
 (('hurto', 3), (1, 1))]

In [206]:
reduced_cases_per_cat_term = cases_per_cat_term.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
reduced_cases_per_cat_term.collect()

[(('robo', 3), (10, 15)),
 (('hurto', 2), (1, 4)),
 (('robo', 2), (2, 3)),
 (('hurto', 3), (1, 1))]

In [207]:
cat_values = reduced_cases_per_cat_term.map(lambda x: (x[0][0], (x[0][1], float(x[1][0]/x[1][1]))))
cat_values.collect()                                            

[('robo', (3, 0.6666666666666666)),
 ('hurto', (2, 0.25)),
 ('robo', (2, 0.6666666666666666)),
 ('hurto', (3, 1.0))]

In [208]:
by_cat = cat_values.groupByKey()
by_cat.collect()

[('hurto', <pyspark.resultiterable.ResultIterable at 0x1133f0a20>),
 ('robo', <pyspark.resultiterable.ResultIterable at 0x1133f0a58>)]

In [209]:
def filter_10p(x):
    data = list(x[1])
    print(data)
    if data[0][0] == 2:
        # es el segundo trimestre
        return (data[1][1] - data[0][1] > 0.10)
    else:
        # es el tercer trimestre primero
        return (data[0][1] - data[1][1] > 0.10)


In [210]:
# filtramos los resultados necesarios para quedarnos con aquellos que hayan tenido la variacion esperada
# nota: se hace el map por formato de salida, se podria hacer con un mapValues
by_cat.filter(filter_10p).map(lambda x: (x[0], list(x[1]))).collect()

[('hurto', [(2, 0.25), (3, 1.0)])]

In [211]:
# b) Tasa de participación de Batman por categoría, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto, y robo de vehículos)

In [212]:
def filter_by_category(x):
    categories = set(['incendio intencional', 'robo', 'hurto', 'robo de vehiculos'])
    return x[4] in categories

In [213]:
cat_filtered = casesRdd.filter(filter_by_category)

In [214]:
key_cat_filtered = cat_filtered.map(lambda x: (x[1],(x[4],1)))

In [215]:
joined = batsignalRdd.leftOuterJoin(key_cat_filtered)
joined.collect()

[(8, (0, ('robo', 1))),
 (80, (1, ('robo', 1))),
 (600, (0, ('robo', 1))),
 (800, (0, ('robo', 1))),
 (1000, (1, ('robo', 1))),
 (6000, (0, ('robo', 1))),
 (7000, (1, ('robo', 1))),
 (8000, (0, ('robo', 1))),
 (9000, (1, ('robo', 1))),
 (10000, (1, ('robo', 1))),
 (96, (1, ('hurto', 1))),
 (1, (1, None)),
 (9, (1, ('robo', 1))),
 (2, (1, ('robo', 1))),
 (10, (0, ('robo', 1))),
 (90, (1, ('robo', 1))),
 (3, (0, ('robo', 1))),
 (4, (0, ('robo', 1))),
 (60, (0, ('robo', 1))),
 (100, (1, ('robo', 1))),
 (700, (1, ('robo', 1))),
 (900, (1, ('robo', 1))),
 (92, (0, ('hurto', 1))),
 (5, (1, ('robo', 1))),
 (93, (0, ('hurto', 1))),
 (6, (0, ('robo', 1))),
 (70, (1, ('robo', 1))),
 (94, (0, ('hurto', 1))),
 (7, (1, ('robo', 1))),
 (95, (0, ('hurto', 1)))]

In [216]:
# Filtramos aquellos cuya categoria es None dado que no los queremos en el resultado final
# K: Category
# V: (sucedio, fue_batman_a_la_senial)
for_stats = joined.filter(lambda x: x[1][1] is not None).map(lambda x: (x[1][1][0], (x[1][1][1], x[1][0])))
for_stats.collect()

[('robo', (1, 0)),
 ('robo', (1, 1)),
 ('robo', (1, 0)),
 ('robo', (1, 0)),
 ('robo', (1, 1)),
 ('robo', (1, 0)),
 ('robo', (1, 1)),
 ('robo', (1, 0)),
 ('robo', (1, 1)),
 ('robo', (1, 1)),
 ('hurto', (1, 1)),
 ('robo', (1, 1)),
 ('robo', (1, 1)),
 ('robo', (1, 0)),
 ('robo', (1, 1)),
 ('robo', (1, 0)),
 ('robo', (1, 0)),
 ('robo', (1, 0)),
 ('robo', (1, 1)),
 ('robo', (1, 1)),
 ('robo', (1, 1)),
 ('hurto', (1, 0)),
 ('robo', (1, 1)),
 ('hurto', (1, 0)),
 ('robo', (1, 0)),
 ('robo', (1, 1)),
 ('hurto', (1, 0)),
 ('robo', (1, 1)),
 ('hurto', (1, 0))]

In [217]:
for_stats.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1])).map(lambda x: (x[0],float(x[1][1]/x[1][0]))).collect()

[('robo', 0.5833333333333334), ('hurto', 0.2)]